In [36]:
import pandas as pd

from logger import Logger
from db import DB

In [37]:
mapping = 'gz2_hart16_clean.txt'

In [38]:
data = pd.read_csv(mapping, sep='\t')
data = data.rename(columns={"name": "gzname"})

In [39]:
data.head(10)

,gzname,type,length,description
0,specobjid,bigint,8,NaN
1,dr8objid,bigint,8,NaN
2,dr7objid,bigint,8,match to the DR7 objID for the corresponding n...
3,ra,real,4,"right ascension [J2000.0], decimal degrees"
4,dec,real,4,"declination [J2000.0], decimal degrees"
5,rastring,varchar,11,"right ascension [J2000.0], sexagesimal"
6,decstring,varchar,11,"declination [J2000.0], sexagesimal"
7,sample,varchar,20,sub-sample identification
8,gz2_class,varchar,20,shorthand string for the most common consensus...
9,total_classifications,int,4,total number of classifications for this galaxy


In [35]:
data.shape

(233, 4)

In [6]:
def translate_type(gztype, length):
    if gztype == 'bigint':
        return 'bigint'
    if gztype == 'real':
        return 'real'
    if gztype == 'int':
        return 'integer'
    if gztype == 'float':
        return 'double precision'
    if gztype == 'real':
        return 'varchar'
    if gztype == 'varchar':
        return f'character varying({length})'
    return f'***** {gztype} not recognized'

In [21]:
fieldlist = [f'{rec.gzname} {translate_type(rec.type, rec.length)}'
             for index, rec in data.iterrows()]

In [22]:
fields = ',\n'.join(fieldlist)
print(fields)

specobjid bigint,
dr8objid bigint,
dr7objid bigint,
ra real,
dec real,
rastring character varying(11),
decstring character varying(11),
sample character varying(20),
gz2_class character varying(20),
total_classifications integer,
total_votes integer,
t01_smooth_or_features_a01_smooth_count integer,
t01_smooth_or_features_a01_smooth_weight double precision,
t01_smooth_or_features_a01_smooth_fraction double precision,
t01_smooth_or_features_a01_smooth_weighted_fraction double precision,
t01_smooth_or_features_a01_smooth_debiased double precision,
t01_smooth_or_features_a01_smooth_flag integer,
t01_smooth_or_features_a02_features_or_disk_count integer,
t01_smooth_or_features_a02_features_or_disk_weight double precision,
t01_smooth_or_features_a02_features_or_disk_fraction double precision,
t01_smooth_or_features_a02_features_or_disk_weighted_fraction double precision,
t01_smooth_or_features_a02_features_or_disk_debiased double precision,
t01_smooth_or_features_a02_features_or_disk_flag in

In [23]:
sql = f"""
-- DROP TABLE IF EXISTS public.gz2data;

CREATE TABLE public.gz2data
(
    {fields},
    PRIMARY KEY (dr7objid)
);

ALTER TABLE IF EXISTS public.gz2data
    OWNER to python;"""


In [24]:
print(sql)


-- DROP TABLE IF EXISTS public.gz2data;

CREATE TABLE public.gz2data
(
    specobjid bigint,
dr8objid bigint,
dr7objid bigint,
ra real,
dec real,
rastring character varying(11),
decstring character varying(11),
sample character varying(20),
gz2_class character varying(20),
total_classifications integer,
total_votes integer,
t01_smooth_or_features_a01_smooth_count integer,
t01_smooth_or_features_a01_smooth_weight double precision,
t01_smooth_or_features_a01_smooth_fraction double precision,
t01_smooth_or_features_a01_smooth_weighted_fraction double precision,
t01_smooth_or_features_a01_smooth_debiased double precision,
t01_smooth_or_features_a01_smooth_flag integer,
t01_smooth_or_features_a02_features_or_disk_count integer,
t01_smooth_or_features_a02_features_or_disk_weight double precision,
t01_smooth_or_features_a02_features_or_disk_fraction double precision,
t01_smooth_or_features_a02_features_or_disk_weighted_fraction double precision,
t01_smooth_or_features_a02_features_or_disk_de

In [25]:
pg = DB()

In [26]:
pg.run_admin("DROP TABLE IF EXISTS public.gz2data")
pg.run_admin(sql)
# pg.run_admin("""ALTER TABLE IF EXISTS public.gz2data
#     ALTER COLUMN dr8objid DROP NOT NULL;""")

In [27]:
pg.run_count('gz2data')

AttributeError: 'psycopg2.extensions.cursor' object has no attribute 'fetch'

In [28]:
names = ','.join(list(data.gzname))
fname = '/home/colin/data/ssd240/zoo2MainSpecz.csv'

In [29]:
uploadsql = f"""COPY gz2data({names})
FROM '{fname}'
DELIMITER ','
CSV HEADER;"""

In [30]:
print(uploadsql)

COPY gz2data(specobjid,dr8objid,dr7objid,ra,dec,rastring,decstring,sample,gz2_class,total_classifications,total_votes,t01_smooth_or_features_a01_smooth_count,t01_smooth_or_features_a01_smooth_weight,t01_smooth_or_features_a01_smooth_fraction,t01_smooth_or_features_a01_smooth_weighted_fraction,t01_smooth_or_features_a01_smooth_debiased,t01_smooth_or_features_a01_smooth_flag,t01_smooth_or_features_a02_features_or_disk_count,t01_smooth_or_features_a02_features_or_disk_weight,t01_smooth_or_features_a02_features_or_disk_fraction,t01_smooth_or_features_a02_features_or_disk_weighted_fraction,t01_smooth_or_features_a02_features_or_disk_debiased,t01_smooth_or_features_a02_features_or_disk_flag,t01_smooth_or_features_a03_star_or_artifact_count,t01_smooth_or_features_a03_star_or_artifact_weight,t01_smooth_or_features_a03_star_or_artifact_fraction,t01_smooth_or_features_a03_star_or_artifact_weighted_fraction,t01_smooth_or_features_a03_star_or_artifact_debiased,t01_smooth_or_features_a03_star_or_ar

In [31]:
pg.run_admin(uploadsql)

In [ ]:
import torch
print(torch.version.cuda)
print(torch.__version__)